# Notebook error analysis 

## Libraries and exception types

#### rules for extracting related libs from crash lines:

    (not included) 1. Uninteresting error types even without knowing if any libraries are involved:
        NameError?, FileNotFoundError, OSError, KeyboardInterrupt?, ModuleNotFoundError?, AssertionError?
    2. compile a list of popular libraries used in data science and ML
        1. if library name is in error value. e.g., 
            TypeError: 'numpy.float64' object is not callable
            AttributeError: module 'torchvision.transforms.v2' has no attribute 'CutMix'
        2. for pattern "----> line_number xx.yy", check if xx is alias of an import library
        ??? 3. similarly, for yy, can check if the property name is from a library. Do not know how.
        4. for some type errors and attribute errors, where to check the object name is from which library.
            AttributeError: 'Sequential' object has no attribute 'predict_classes'
            TypeError: 'AxesSubplot' object is not subscriptable
        ??? 5. for ValueError, certain keywords in error values indicates library names
            broadcast, array, shape -> numpy

In [6]:
# lib2 parsing config
import pickle

#error_type_ignore = ["nameerror", "filenotfounderror", "oserror", "keyboardinterrupt", "modulenotfounderror", "assertionerror"]
lib_names = ["pandas","numpy", "matplotlib","sklearn","seaborn","tensorflow","torch","tqdm","random","xgboost",
             "scipy","plotly","keras","cv2","PIL","math","lightgbm","nltk","torchvision","collections",
             "transformers","catboost","statsmodels","imblearn","wordcloud"]
with open('lib_classes.pickle', 'rb') as f:
    lib_classes_dict = pickle.load(f)


### 1. nbs from github - the stack v1

https://huggingface.co/datasets/bigcode/the-stack-dedup/tree/main/data/jupyter-notebook


In [2]:
# read in all errors from the file
import pandas as pd
import util

df2_err_builtin_exps = pd.read_excel('C:/Users/yirwa29/Downloads/Dataset-Nb/nberror_g_all_p4_exception_types.xlsx')
df2_err_builtin_exps.head()

,fname,ename,evalue,traceback,lib
0,00000-101-cookie-clicker-v2-checkpoint.ipynb,keyboardinterrupt,NaN,['--------------------------------------------...,"urllib3,selenium"
1,00000-1012-demand-forecasting-data-prep-from-s...,libcustomerrors,"FATAL: no pg_hba.conf entry for host ""75.166....",['--------------------------------------------...,psycopg2
2,00000-1017-heatmapseq2seq.ipynb,valueerror,"x and y must have same first dimension, but ha...",['--------------------------------------------...,matplotlib
3,00000-1023-crawler-20190515-20190516.ipynb,connectionerror,HTTPSConnectionPool(host='www.backpackers.com....,['--------------------------------------------...,"requests,urllib3"
4,00000-1033-peer-solution-predicting-survival-t...,libcustomerrors,<urlopen error [Errno 11001] getaddrinfo failed>,['--------------------------------------------...,pandas


In [2]:
# lib percentage from first general lib extraction -> util.extract_lib
len(df2_err_builtin_exps[~df2_err_builtin_exps["lib"].isnull()])/len(df2_err_builtin_exps)

0.35296669615239423

### 1.1 Extract imported libraries and their alias used in the error notebook dataset

In [5]:
import imports_parser
import pickle
import pandas as pd

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
res = imports_parser.get_imports_nbs_static(path_err_nbs+"/nbs", imports_parser.get_imports_line_all)
res_pd = pd.DataFrame.from_dict(res)
res_pd["lib_alias"] = res_pd.imports.apply(imports_parser.get_lib_alias)
res_pd.to_excel(path_err_nbs+"/imports_all_info.xlsx", index=False, engine="xlsxwriter")

Unexpected error converting to json 00279-3344-datasets.ipynb


### 1.1.1 filter only the ones using the selected libraries, based on extracted all imports from error nbs

In [2]:
import pandas as pd
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
df_imports = pd.read_excel(path_err_nbs + '/imports_all_info.xlsx')

In [3]:
df_imports["lib_alias"] = df_imports.lib_alias.apply(eval)
df_imports_filtered = df_imports.loc[df_imports.lib_alias.apply(lambda imports: any(util.simple_lib_parser(imp[0]) in lib_names for imp in imports))]
print("{0:.2%} of all the GitHub errors notebooks containing errors use the selected libraries".format(len(df_imports_filtered)/len(df_imports)))

81.06% of all the GitHub errors notebooks containing errors use the selected libraries


In [4]:
df_imports_filtered.to_excel(path_err_nbs+"/imports_all_info_filtered_ML.xlsx", index=False, engine="xlsxwriter")

### 1.2 Extract related libraries from crash lines in tracebacks

The second attempt to extracting libs.  -> util.extract_lib_2

In [1]:
import pandas as pd
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
df_imports = pd.read_excel(path_err_nbs + '/imports_all_info.xlsx')
df2_err_builtin_exps = pd.read_excel('C:/Users/yirwa29/Downloads/Dataset-Nb/nberror_g_all_p4_exception_types.xlsx')
df2_err_builtin_exps.head()

,fname,ename,evalue,traceback,lib
0,00000-101-cookie-clicker-v2-checkpoint.ipynb,keyboardinterrupt,NaN,['--------------------------------------------...,"urllib3,selenium"
1,00000-1012-demand-forecasting-data-prep-from-s...,libcustomerrors,"FATAL: no pg_hba.conf entry for host ""75.166....",['--------------------------------------------...,psycopg2
2,00000-1017-heatmapseq2seq.ipynb,valueerror,"x and y must have same first dimension, but ha...",['--------------------------------------------...,matplotlib
3,00000-1023-crawler-20190515-20190516.ipynb,connectionerror,HTTPSConnectionPool(host='www.backpackers.com....,['--------------------------------------------...,"requests,urllib3"
4,00000-1033-peer-solution-predicting-survival-t...,libcustomerrors,<urlopen error [Errno 11001] getaddrinfo failed>,['--------------------------------------------...,pandas


In [4]:
df2_err_builtin_exps["lib2"] = df2_err_builtin_exps.apply(util.extract_lib_2, lib_names=lib_names, df_imports=df_imports, lib_classes_dict=lib_classes_dict, axis=1)

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

In [5]:
1-len(df2_err_builtin_exps[df2_err_builtin_exps.lib.isnull()&df2_err_builtin_exps.lib2.isnull()])/len(df2_err_builtin_exps)

0.45626440246327904

In [ ]:
df2_err_builtin_exps["lib_parsed"] = df2_err_builtin_exps["lib2"].fillna(df2_err_builtin_exps["lib"]).map(util.simple_lib_parser)
df2_err_builtin_exps.to_excel(path_err_nbs+"/nberror_g_all_p4_exception_types_lib_parsed.xlsx", index=False, engine='xlsxwriter')

### 1.3 Analyzing crashed libraries in relation to exception types

In [28]:
import pandas as pd
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
df2_err_builtin_exps = pd.read_excel(path_err_nbs+"/nberror_g_all_p4_exception_types_lib_parsed.xlsx")
sum(~df2_err_builtin_exps["lib_parsed"].isna())/len(df2_err_builtin_exps)

0.456264402463279

In [2]:
df2_err_builtin_exps.lib_parsed.value_counts()[:10]

lib_parsed
pandas        15277
numpy         14134
tensorflow    10156
torch          8294
sklearn        5491
matplotlib     3585
scipy          1247
ipython         980
seaborn         848
cv2             654
Name: count, dtype: int64

### 1.3.1 lib-exception relationship  -  all libs

In [3]:
# lib-exception relationship  -  all libs
for builtin_exp in df2_err_builtin_exps.ename.value_counts().index:
    df2_err_builtin_exp = df2_err_builtin_exps[df2_err_builtin_exps["ename"]==builtin_exp]
    libs_n = df2_err_builtin_exp.lib_parsed.value_counts()
    lib_percent = len(df2_err_builtin_exp[~df2_err_builtin_exp["lib_parsed"].isnull()])/len(df2_err_builtin_exp)
    n = min(3, len(libs_n))
    print("For {0}({1}), {2:.2%} are with libraries, the top {3} are:".format(builtin_exp, len(df2_err_builtin_exp),lib_percent, n))
    for i in range(n):
        print("\t{0:<12} {1:>12} samples".format(libs_n.index[i], libs_n.values[i]))

For nameerror(36962), 15.44% are with libraries, the top 3 are:
	numpy                1663 samples
	pandas               1122 samples
	sklearn               598 samples
For typeerror(23563), 36.93% are with libraries, the top 3 are:
	numpy                3225 samples
	pandas               1539 samples
	torch                 980 samples
For keyboardinterrupt(18781), 82.01% are with libraries, the top 3 are:
	tensorflow           3886 samples
	torch                2460 samples
	numpy                2069 samples
For valueerror(18143), 68.72% are with libraries, the top 3 are:
	pandas               2385 samples
	sklearn              2311 samples
	numpy                2183 samples
For attributeerror(12924), 63.53% are with libraries, the top 3 are:
	numpy                1982 samples
	pandas               1806 samples
	torch                1459 samples
For libcustomerrors(12804), 73.94% are with libraries, the top 3 are:
	tensorflow           1448 samples
	pandas                863 samples
	

### 1.3.2 lib-exception relationship  -  the selected (from kaggle) popular libs

In [21]:
# lib-exception relationship  -  popular libs
df2_err_builtin_exps["lib_parsed_pop"] = df2_err_builtin_exps['lib_parsed'].apply(lambda i: i if i in lib_names else None)

for builtin_exp in df2_err_builtin_exps.ename.value_counts().index:
    df2_err_builtin_exp = df2_err_builtin_exps[df2_err_builtin_exps["ename"]==builtin_exp]
    
    libs_n = df2_err_builtin_exp.lib_parsed_pop.value_counts()
    lib_percent = len(df2_err_builtin_exp[~df2_err_builtin_exp["lib_parsed_pop"].isnull()])/len(df2_err_builtin_exp)
    n = min(3, len(libs_n))
    print("For {0}({1}), {2:.2%} are with libraries, top {3}:".format(builtin_exp, len(df2_err_builtin_exp),lib_percent, n))
    for i in range(n):
        print("\t{0:<12} {1:>12} samples".format(libs_n.index[i], libs_n.values[i]))

For nameerror(36962), 14.74% are with libraries, top 3:
	numpy                1663 samples
	pandas               1122 samples
	sklearn               598 samples
For typeerror(23563), 32.16% are with libraries, top 3:
	numpy                3225 samples
	pandas               1539 samples
	torch                 980 samples
For keyboardinterrupt(18781), 60.92% are with libraries, top 3:
	tensorflow           3886 samples
	torch                2460 samples
	numpy                2069 samples
For valueerror(18143), 59.93% are with libraries, top 3:
	pandas               2385 samples
	sklearn              2311 samples
	numpy                2183 samples
For attributeerror(12924), 58.67% are with libraries, top 3:
	numpy                1982 samples
	pandas               1806 samples
	torch                1459 samples
For libcustomerrors(12804), 32.11% are with libraries, top 3:
	tensorflow           1448 samples
	pandas                863 samples
	numpy                 683 samples
For syntaxerro

### derived results==>

Selection criterion:

    exclude exception types: ["nameerror", "keyboardinterrupt", "modulenotfounderror", "filenotfounderror", "importerror", "oserror", "memoryerror", "ioerror"]
    total amount of the errors in each exception type needs to be greater than 2000
    >=30% of the errors in each exception type are from the library list
    
    
=> typeerror, valueerror, attributeerror, libcustomerrors, keyerror, runtimeerror

### 1.3.3 lib-exception relationship  -  the selected (from kaggle) popular libs + on filtered nbs(that use the selected libraries)

In [1]:
# lib-exception relationship  -  popular libs + on filtered nbs(that use the selected libraries)
import pandas as pd
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
df2_err_builtin_exps = pd.read_excel(path_err_nbs+"/nberror_g_all_p4_exception_types_lib_parsed.xlsx")
df_imports_filtered = pd.read_excel(path_err_nbs+"/imports_all_info_filtered_ML.xlsx")

In [2]:
df2_err_builtin_exps_filtered = pd.merge(df2_err_builtin_exps, df_imports_filtered, on='fname', how='inner').drop(columns=["imports","lib_alias"])
sum(~df2_err_builtin_exps_filtered["lib_parsed"].isna())/len(df2_err_builtin_exps_filtered)

0.5093050534706168

In [3]:
df2_err_builtin_exps_filtered.lib_parsed.value_counts()[:10]

lib_parsed
pandas        15082
numpy         12985
tensorflow     9926
torch          7645
sklearn        5454
matplotlib     3501
scipy          1213
seaborn         837
ipython         734
cv2             649
Name: count, dtype: int64

In [4]:
builtin_exps_excluded = ["nameerror", "keyboardinterrupt", "modulenotfounderror", "filenotfounderror", "importerror", "oserror", "memoryerror", "ioerror"]
lib_percent_cutoff = 0.3
err_count_cutoff = 2000

In [8]:
df2_err_builtin_exps_filtered["lib_parsed_pop"] = df2_err_builtin_exps_filtered['lib_parsed'].apply(lambda i: i if i in lib_names else None)

for builtin_exp in df2_err_builtin_exps_filtered.ename.value_counts().index:
    # cutoff 1
    if builtin_exp in builtin_exps_excluded:
        continue
    df2_err_builtin_exp = df2_err_builtin_exps_filtered[df2_err_builtin_exps_filtered["ename"]==builtin_exp]
    # cutoff 2
    if len(df2_err_builtin_exp) < err_count_cutoff:
        continue
    libs_n = df2_err_builtin_exp.lib_parsed_pop.value_counts()
    lib_percent = len(df2_err_builtin_exp[~df2_err_builtin_exp["lib_parsed_pop"].isnull()])/len(df2_err_builtin_exp)
    # cutoff 3
    if lib_percent < lib_percent_cutoff:
        continue
    n = min(3, len(libs_n))
    print("For {0}({1}), {2:.2%} are with libraries, top {3}:".format(builtin_exp, len(df2_err_builtin_exp),lib_percent, n))
    for i in range(n):
        print("\t{0:<12} {1:>12} samples".format(libs_n.index[i], libs_n.values[i]))

For typeerror(17309), 38.15% are with libraries, top 3:
	numpy                2421 samples
	pandas               1525 samples
	torch                 858 samples
For valueerror(16145), 66.63% are with libraries, top 3:
	pandas               2359 samples
	sklearn              2304 samples
	numpy                2165 samples
For attributeerror(10486), 65.95% are with libraries, top 3:
	pandas               1759 samples
	numpy                1740 samples
	torch                1174 samples
For libcustomerrors(9637), 41.33% are with libraries, top 3:
	tensorflow           1387 samples
	pandas                849 samples
	numpy                 671 samples
For keyerror(6515), 64.97% are with libraries, top 3:
	pandas               3660 samples
	numpy                 290 samples
	tensorflow             62 samples
For runtimeerror(3523), 72.89% are with libraries, top 3:
	torch                1690 samples
	numpy                 467 samples
	tensorflow            154 samples


In [17]:
# printing tracebacks for certain exception type + related to selected libraries
import util

n = 10
print_ename = "runtimeerror"
print_tb = df2_err_builtin_exps_filtered["traceback"][(df2_err_builtin_exps_filtered["ename"]==print_ename)&
                                                      any(df2_err_builtin_exps_filtered["lib_parsed"].isin(lib_names))]
n = min(n,len(print_tb))
for i in range(n):
    util.print_traceback(print_tb.iloc[i])

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<timed eval> in <module>

<ipython-input-36-8f18ffca7336> in train_bert(model, train_loader, epochs)
     28             epoch_loss += loss
     29 
---> 30             loss.backward()
     31             torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
     32             optimizer.step()

~/cs224u/venv/lib/python3.6/site-packages/torch/tensor.py in backward(self, gradient, retain_graph, create_graph)
    164                 products. Defaults to ``False``.
    165         """
--> 166         torch.autograd.backward(self, gradient, retain_graph, create_graph)
    167 
    168     def register_hook(self, hook):

~/cs224u/venv/lib/python3.6/site-packages/torch/autograd/__init__.py in backward(tensors, grad_tensors, retain_graph, create_graph,

### 2. nbs from kaggle

### 2.1 Extract imported libraries and their alias used in the error notebook dataset

In [ ]:
import imports_parser
import pickle

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error"
res = imports_parser.get_imports_nbs_static(path_err_nbs, imports_parser.get_imports_line_all)
res_pd = pd.DataFrame.from_dict(res)
res_pd["lib_alias"] = res_pd.imports.apply(imports_parser.get_lib_alias)

In [157]:
res_pd.to_excel(path_err_nbs+"/imports_all_info.xlsx", index=False, engine="xlsxwriter")
res_pd

,fname,imports,lib_alias
0,aadyac_ingenium-level3.ipynb,"{(, numpy, np), (tensorflow.keras.utils, plot_...","[[numpy, np], [tensorflow, plot_model], [cv2, ..."
1,abaojiang_eda-on-game-progress.ipynb,"{(typing, Any,, ), (, warnings, ), (, numpy, n...","[[typing, Any,], [warnings, warnings], [numpy,..."
2,abdallahelsayed22_image-segmentation-u-net.ipynb,"{(, cv2, ), (keras.layers, Input,, ), (, imgau...","[[cv2, cv2], [keras, Input,], [imgaug, iaa], [..."
3,abdallahwagih_plant-stress-identification-acc-...,"{(tensorflow.keras.preprocessing.image, ImageD...","[[tensorflow, ImageDataGenerator], [tensorflow..."
4,abdelrahmanmuhsen_semseg-tests.ipynb,"{(, warnings, ), (, cv2, ), (keras.callbacks, ...","[[warnings, warnings], [cv2, cv2], [keras, Mod..."
...,...,...,...
4344,yeohhanyi_cirrhosis-outcomes.ipynb,"{(sklearn.preprocessing, LabelEncoder,, ), (, ...","[[sklearn, LabelEncoder,], [numpy, np], [xgboo..."
4345,yeohqiwei_credit-card-fraud-transaction-classi...,"{(, numpy, np), (math, radians,, ), (imblearn....","[[numpy, np], [math, radians,], [imblearn, ADA..."
4346,zainabmuhammad_house-prices-prediction-ip-proj...,"{(, numpy, np), (scipy, stats, ), (, seaborn, ...","[[numpy, np], [scipy, stats], [seaborn, sns], ..."
4347,zakirkhanaleemi_gemini-api-entrant-notebook.ipynb,"{(, pathlib, ), (, google.ai.generativelanguag...","[[pathlib, pathlib], [google, glm], [IPython, ..."


### 2.2 Extract related libraries from crash lines in tracebacks

In [1]:
import pandas as pd
import pickle
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error"
df_err = pd.read_excel(path_err_nbs + '/nberror_k_p3.xlsx')
df_imports = pd.read_excel(path_err_nbs + '/imports_all_info.xlsx')

In [3]:
df_err["lib2"] = df_err.apply(util.extract_lib_2, lib_names=lib_names, df_imports=df_imports, lib_classes_dict=lib_classes_dict, axis=1)

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

In [45]:
1-len(df_err[df_err.lib.isnull()&df_err.lib2.isnull()])/len(df_err)

0.6208677685950413

In [4]:
df_err["lib_parsed"] = df_err["lib2"].fillna(df_err["lib"]).map(util.simple_lib_parser)
df_err.to_excel(path_err_nbs+"/nberror_k_all_p3_lib_parsed.xlsx", index=False, engine='xlsxwriter')

### 2.3 Analyzing the extract crash-related libraries

In [7]:
import pandas as pd
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error"
df_err = pd.read_excel(path_err_nbs+"/nberror_k_all_p3_lib_parsed.xlsx")
sum(~df_err.lib_parsed.isna())/len(df_err)

0.6208677685950413

In [130]:
import util

for i in range(90,100):
    util.print_traceback(df_err[df_err.lib.isnull()].traceback.iloc[i])

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[70], line 3
      1 inputs = Input((input_shape))
      2 # convolutional layers
----> 3 conv_1 = conv2D(32, kernel_size=(3, 3),activation='relu') (inputs)
      4 maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
      5 conv_2 = conv2D(64, kernel_size=(3, 3),activation='relu') (maxp_1)

NameError: name 'conv2D' is not defined
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[71], line 3
      1 #plot the model
      2 from tensorflow.keras.utils import plot_model
----> 3 plot_model(model)

NameError: name 'model' is not defined
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[19], line 15
      1 image_gen = Im